# NLP Sentiment Analysis - Interpretation of Results

### Make imports and load data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from joblib import load
import spacy
import pandas as pd
from sklearn.metrics import confusion_matrix

model = load('final_model.joblib')
vectorizer = spacy.load('en_core_web_lg')
mean_vec = load('mean_vec.joblib')

In [2]:
data = pd.read_csv('data/all-data-v2.csv', sep=';', encoding='latin-1')

def remove_quot_mark(string):
    if string[0] == ' ':
        string = string[1:]
    if string[0] == '"':
        string = string[1:]
    if string[-1] =='"':
        string = string[:-1]
    return string
        
data['sentence'] = data['sentence'].apply(remove_quot_mark)
data['label'] = data.apply(lambda row: int(row['sentiment']=='positive') - int(row['sentiment']=='negative'), axis=1)
data.head()

,sentiment,sentence,label
0,neutral,"According to Gran, the company has no plans to...",0
1,neutral,Technopolis plans to develop in stages an area...,0
2,negative,The international electronic industry company ...,-1
3,positive,With the new production plant the company woul...,1
4,positive,According to the company 's updated strategy f...,1


### Define function to transform data into Word2Vec representation

In [3]:
def transform_w2v_test(X_test, vectorizer, mean_vec):
    
    with vectorizer.disable_pipes():
        test_vectors = np.array([vectorizer(text).vector for text in X_test])

    X_test = test_vectors - mean_vec
    return X_test

### Evaluate "out-of-sample" performance

In [4]:
y_pred = model.predict(transform_w2v_test(data['sentence'], vectorizer, mean_vec))
cm = confusion_matrix(data['label'],y_pred, labels=[1,0,-1])
precision = np.trace(cm)/np.sum(cm)

print('Confusion Matrix:')
print(cm)
print('\n\nEstimated precision:' + str(precision))

Confusion Matrix:
[[1241  105   17]
 [1663 1178   38]
 [ 472   35   97]]


Estimated precision:0.5191910854312836


The "out-of-sample" precision is rather poor as only approx. 50% of the data is classified correctly. There are more than 60% neutral data, so classifying all data as neutral would yield a better precision. The neutral data could be considered as noise. The following results investigate the perfomance of the procedure with only positive and negative data.

## Evaluate model trained with positive and negative data only

In [5]:
model = load('final_model_pos_neg.joblib')
vectorizer = load('vectorizer_pos_neg.joblib')

data2 = data[data['sentiment'] != 'neutral']

y_pred = model.predict(vectorizer.transform(data2['sentence']))
cm = confusion_matrix(data2['label'],y_pred, labels=[1,-1])
precision = np.trace(cm)/np.sum(cm)

print('Confusion Matrix:')
print(cm)
print('\n\nEstimated precision:' + str(precision))

Confusion Matrix:
[[1271   92]
 [ 447  157]]


Estimated precision:0.7259786476868327


In [7]:
val_counts = data2['label'].value_counts()

print('Number of positive sentences: ' + str(val_counts[1]) + '\n' 
      + 'Number of negative sentences: ' + str(val_counts[-1]))

Number of positive sentences: 1363
Number of negative sentences: 604
